In [2]:
pip install pandas scikit-learn tensorflow keras-tuner matplotlib


Note: you may need to restart the kernel to use updated packages.


In [10]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras

# CSV 파일 경로 설정 (자신의 CSV 파일 경로로 변경)
file_path = file_path = "C:/Users/water/OneDrive/바탕 화면/iris-classification/wine.csv"
# 데이터 불러오기
df = pd.read_csv(file_path)
# 데이터 확인
print("데이터 미리보기:")
print(df.head()) #df.head()괄호 안에 숫자만큼 행 출력

데이터 미리보기:
   Wine  Alcohol  Malic.acid   Ash   Acl   Mg  Phenols  Flavanoids  \
0     1    14.23        1.71  2.43  15.6  127     2.80        3.06   
1     1    13.20        1.78  2.14  11.2  100     2.65        2.76   
2     1    13.16        2.36  2.67  18.6  101     2.80        3.24   
3     1    14.37        1.95  2.50  16.8  113     3.85        3.49   
4     1    13.24        2.59  2.87  21.0  118     2.80        2.69   

   Nonflavanoid.phenols  Proanth  Color.int   Hue    OD  Proline  
0                  0.28     2.29       5.64  1.04  3.92     1065  
1                  0.26     1.28       4.38  1.05  3.40     1050  
2                  0.30     2.81       5.68  1.03  3.17     1185  
3                  0.24     2.18       7.80  0.86  3.45     1480  
4                  0.39     1.82       4.32  1.04  2.93      735  


In [12]:
# 컬렴명 확인
df.columns

Index(['Wine', 'Alcohol', 'Malic.acid', 'Ash', 'Acl', 'Mg', 'Phenols',
       'Flavanoids', 'Nonflavanoid.phenols', 'Proanth', 'Color.int', 'Hue',
       'OD', 'Proline'],
      dtype='object')

In [14]:
# X, y 나누기
X = df.drop("Wine", axis=1)
y = df["Wine"] - 1  # Wine이 1, 2, 3이라면 0, 1, 2로 조정

In [20]:
# 원핫 인코딩
from tensorflow.keras.utils import to_categorical
y_encoded = to_categorical(y)

In [24]:
# 정규화 (스케일링)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [26]:
# 학습/테스트 나누기
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42)

In [30]:
from tensorflow import keras
from keras import layers
import keras_tuner as kt

def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Input(shape=(X.shape[1],)))

    # 은닉층 수 및 뉴런 수 튜닝
    for i in range(hp.Int('num_layers', 1, 3)):
        model.add(layers.Dense(
            units=hp.Int(f'units_{i}', min_value=32, max_value=128, step=32),
            activation=hp.Choice('activation', ['relu', 'tanh'])
        ))

    # 출력층: 클래스 수 = 3
    model.add(layers.Dense(3, activation='softmax'))

    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Float('lr', 1e-4, 1e-2, sampling='log')
        ),
        loss='categorical_crossentropy',  
        metrics=['accuracy']
    )
    return model


In [32]:
#하이퍼 파라미터 탐색
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=2,
    directory='tuner_results',
    project_name='wine_classification_oh'
)

# 탐색 실행
tuner.search(X_train, y_train, epochs=50, validation_split=0.2, verbose=1)


Trial 10 Complete [00h 00m 16s]
val_accuracy: 0.8965517282485962

Best val_accuracy So Far: 1.0
Total elapsed time: 00h 02m 09s


In [36]:
# 최적 모델 가져오기
best_model = tuner.get_best_models(num_models=1)[0]

# 테스트셋 정확도 평가
loss, acc = best_model.evaluate(X_test, y_test)
print(f" Best Test Accuracy: {acc:.4f}")
tuner.results_summary()


C:\Users\water\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.9711 - loss: 0.0846 
 Best Test Accuracy: 0.9722
Results summary
Results in tuner_results\wine_classification_oh
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 00 summary
Hyperparameters:
num_layers: 2
units_0: 32
activation: relu
lr: 0.0010872891569665949
units_1: 32
Score: 1.0

Trial 01 summary
Hyperparameters:
num_layers: 3
units_0: 32
activation: tanh
lr: 0.00306183539339099
units_1: 128
units_2: 32
Score: 1.0

Trial 02 summary
Hyperparameters:
num_layers: 2
units_0: 96
activation: tanh
lr: 0.00535886478745331
units_1: 96
units_2: 64
Score: 1.0

Trial 03 summary
Hyperparameters:
num_layers: 2
units_0: 128
activation: tanh
lr: 0.0004976728854421018
units_1: 32
units_2: 64
Score: 1.0

Trial 05 summary
Hyperparameters:
num_layers: 3
units_0: 64
activation: tanh
lr: 0.002483282483546569
units_1: 32
units_2: 128
Score: 1.0

Trial 06 summary
Hyperparameters:
num_layers: 3
units_0: 32
activation: tanh
lr: 0

In [38]:
best_model = tuner.get_best_models(num_models=1)[0]


C:\Users\water\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [40]:
loss, acc = best_model.evaluate(X_test, y_test)
print(f"Test Accuracy: {acc:.4f}")


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.9711 - loss: 0.0846 
Test Accuracy: 0.9722


In [44]:
import numpy as np

# 예측 결과 (확률)
pred_probs = best_model.predict(X_test)

# 실제 분류 결과 (0, 1, 2 중 하나)
pred_classes = np.argmax(pred_probs, axis=1)

# 실제 정답
true_classes = np.argmax(y_test, axis=1)


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step


In [46]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# 예측 확률 → 클래스 예측 (0, 1, 2 중 하나)
y_pred_probs = best_model.predict(X_test)
y_pred_class = np.argmax(y_pred_probs, axis=1)

# 정답 라벨도 원핫 → 정수로 변환
y_test_class = np.argmax(y_test, axis=1)

# 🔍 결과 출력
print(" Classification Report")
print(classification_report(y_test_class, y_pred_class))

print(" Confusion Matrix")
print(confusion_matrix(y_test_class, y_pred_class))


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
 Classification Report
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        14
           1       1.00      0.93      0.96        14
           2       0.89      1.00      0.94         8

    accuracy                           0.97        36
   macro avg       0.96      0.98      0.97        36
weighted avg       0.98      0.97      0.97        36

 Confusion Matrix
[[14  0  0]
 [ 0 13  1]
 [ 0  0  8]]
